# Mini Project: Dermatologist AI

## Introduction
In this notebook we'll try to design a model that can visually diagnose melanoma, the deadliest form of skin cancer. Here the model will try to distinguish this malignant skin tumor from two types of benign lesions (nevi and seborrheic keratoses)

The data and objective are pulled from the 2017 ISIC Challenge on Skin Lesion Analysis Towards Melanoma Detection. As part of the challenge, participants were tasked to design an algorithm to diagnose skin lesion images as one of three different skin diseases (melanoma, nevus, or seborrheic keratosis). In this project, you will create a model to generate your own predictions.


## Evaluation
Inspired by the ISIC challenge, your algorithm will be ranked according to three separate categories.

### Category 1: ROC AUC for Melanoma Classification
In the first category, we will gauge the ability of your CNN to distinguish between malignant melanoma and the benign skin lesions (nevus, seborrheic keratosis) by calculating the area under the receiver operating characteristic curve (ROC AUC) corresponding to this binary classification task.

If you are unfamiliar with ROC (Receiver Operating Characteristic) curves and would like to learn more, you can check out the documentation in scikit-learn or read this Wikipedia article.

The top scores (from the ISIC competition) in this category can be found in the image below.

### Category 2: ROC AUC for Melanocytic Classification
All of the skin lesions that we will examine are caused by abnormal growth of either melanocytes or keratinocytes, which are two different types of epidermal skin cells. Melanomas and nevi are derived from melanocytes, whereas seborrheic keratoses are derived from keratinocytes.

In the second category, we will test the ability of your CNN to distinuish between melanocytic and keratinocytic skin lesions by calculating the area under the receiver operating characteristic curve (ROC AUC) corresponding to this binary classification task.

The top scores in this category (from the ISIC competition) can be found in the image below.

### Category 3: Mean ROC AUC
In the third category, we will take the average of the ROC AUC values from the first two categories.

The top scores in this category (from the ISIC competition) can be found in the image below.

In [1]:
ROOT = "/kaggle/input/derma-diseases/dataset"
!ls {ROOT}

test  train  validation


In [6]:
# importing libraries
import os
import cv2
import torch
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
from glob import glob
from tqdm import tqdm
import torch.nn as nn
from torch import optim
from PIL import ImageFile
from PIL import Image

## Calculating Mean and STD for our Dataset

In [7]:


data_imgs = np.array(glob(ROOT+"/*/*/*"))

average_im = torch.zeros(3)
average_im_square = torch.zeros(3)
for idx in tqdm(range(len(data_imgs))):
    img = Image.open(data_imgs[idx])
    img = transforms.Resize(224)(img)
    output = transforms.ToTensor()(img)
    average_im += torch.mean(output.view(3,-1), dim=1)
    average_im_square += torch.mean(output.view(3,-1)**2, dim=1)
average_im /= len(data_imgs)
std_im = np.sqrt(average_im_square/len(data_imgs) - average_im**2)
average_im
print(average_im)
print(std_im)

100%|██████████| 1403/1403 [00:08<00:00, 156.28it/s]

tensor([0.6908, 0.5535, 0.5109])
tensor([0.1494, 0.1543, 0.1641])


## Specify Transforms and Loaders

In [9]:
batch_size = 64

train_transforms = transforms.Compose([
    transforms.Resize((450, 450)), 
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean = average_im, std = std_im)
])


test_transforms = transforms.Compose([
    transforms.Resize((450,450)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = average_im, std = std_im)
    ])

# Specifying our training, validation, and test sets
train_data = datasets.ImageFolder(ROOT + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(ROOT + '/test', transform=test_transforms)
valid_data =  datasets.ImageFolder(ROOT + '/validation', transform=test_transforms)

# prepare the data loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)

dataset_size = len(train_data+test_data+valid_data)
classes = train_data.classes
number_classes = len(train_data.classes)
print("data set size: ", dataset_size)
print("Number of classes: ", number_classes)
print("Classes: ", classes)

data set size:  1403
Number of classes:  3
Classes:  ['melanoma', 'nevus', 'seborrheic_keratosis']


In [10]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()

In [11]:
# Load the pretrained model
# define VGG16 model
ResNet = models.resnet152(pretrained=True)
if use_cuda:
    ResNet.cuda()
ResNet

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [38]:
# Building the network

#freeze the model calssifier
for param in  ResNet.parameters():
    param.requires_grad = False

from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([                           
                          ('fc1', nn.Linear(2048, 1024)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.2)),
                          ('fc2', nn.Linear(1024, 512)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.2)),
                          ('fc3', nn.Linear(512, number_classes)),
                          ('output', nn.LogSoftmax(dim=1))]))

ResNet.fc = classifier
if use_cuda:
    ResNet = ResNet.cuda()
ResNet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [39]:
# apply normal distribution rule ro initialize the weights
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
       values taken from a normal distribution.'''
    
    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        # m.weight.data shoud be taken from a normal distribution
        n = m.in_features
        y = 1.0/np.sqrt(n)    
        m.weight.data.normal_(0, y)
        # m.bias.data should be 0
        m.bias.data.fill_(0)


In [40]:
# Initializing the weights
ResNet.fc.apply(weights_init_normal)

Sequential(
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=3, bias=True)
  (output): LogSoftmax()
)

In [41]:
#specify lr
lr = 0.01 * batch_size/256
# Specify Loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.SGD(ResNet.fc.parameters(), lr=0.0005, momentum=0.9)

In [42]:
# the following import is required for training to be robust to truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, trainloader, testloader, validloader , model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(trainloader):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # clear the gradient
            optimizer.zero_grad()
            output = model(data)
            
            ## find the loss and update the model parameters accordingly
            loss = criterion(output, target)
            loss.backward()
            # Perform the optimizer step
            optimizer.step()
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            train_loss += loss.item() * data.size(0)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(validloader):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()           
            # Pass data acrss the network
            output = model(data)
            loss = criterion(output, target)
            ## update the average validation loss
            valid_loss += loss.item() * data.size(0)
        
        # Calculate the average losses
        train_loss = train_loss/len(trainloader)
        valid_loss = valid_loss/len(validloader)
    
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            valid_loss_min = valid_loss
            # print the decremnet in the validation
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min, 
            valid_loss))
            torch.save(model.state_dict(), save_path)
            
    # return trained model
    return model


In [43]:
def test(testloader, model, criterion, use_cuda):
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(testloader):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))



In [ ]:
# Train our model
model = train(50, trainloader, testloader, validloader,
                       ResNet, optimizer, criterion, use_cuda, 'dermatologist_ai.pt')


Epoch: 1 	Training Loss: 69.371145 	Validation Loss: 50.446721
Validation loss decreased (50.446721 --> 50.446721).  Saving model ...
Epoch: 2 	Training Loss: 62.783361 	Validation Loss: 46.308066
Validation loss decreased (46.308066 --> 46.308066).  Saving model ...
Epoch: 3 	Training Loss: 58.570321 	Validation Loss: 43.937645
Validation loss decreased (43.937645 --> 43.937645).  Saving model ...
Epoch: 4 	Training Loss: 53.022848 	Validation Loss: 41.047692
Validation loss decreased (41.047692 --> 41.047692).  Saving model ...
Epoch: 5 	Training Loss: 50.582325 	Validation Loss: 40.328621
Validation loss decreased (40.328621 --> 40.328621).  Saving model ...
Epoch: 6 	Training Loss: 49.176722 	Validation Loss: 38.300821
Validation loss decreased (38.300821 --> 38.300821).  Saving model ...
Epoch: 7 	Training Loss: 48.122109 	Validation Loss: 37.536700
Validation loss decreased (37.536700 --> 37.536700).  Saving model ...


In [ ]:
# load the model that got the best validation accuracy
model.load_state_dict(torch.load('dermatologist_ai.pt'))

In [ ]:
# test our model on the testin set
test(testloader, model, criterion, use_cuda)

In [ ]:
# Predict a given image
def predict(img_path):
    # load the image and return the predicted breed
    # pre_process the image
    t_batch = process_img(img_path, 224)
    Resnet.eval()
    t_batch = t_batch.cuda()
    output = model_transfer(t_batch)
    _, pred = torch.max(output, 1)    
    class_name = classes[int(pred)]
    return class_name